### OT2 script test environment

In [4]:
class S300:
    def __init__(self):
        # Initialize an instance variable to store volume
        self.current_volume = 0  # Placeholder for a volume value
    
    def aspirate(self, volume, location):
        self.current_volume += volume  # Store the aspirated volume
        print(f"Aspirating {volume}uL from {location}")
        
    def dispense(self, volume, location):
        self.current_volume -= volume
        print(f"Dispensing {volume}uL to {location}")
    
    def mix(self, times, volume, location):
        print(f"Mixing {times} times with {volume}uL at {location}")
    
    def pick_up_tip(self, tip_location):
        print(f"Picking up tip at {tip_location}")
    
    def blow_out(self, tip_location):
        self.current_volume = 0
        print(f"Blowing Buffer from tip at {tip_location}")
    def return_tip(self):
        print(f"Returning Tip")

# Dummy objects for plate and tip locations
class Well:
    def __init__(self, well_id):
        self.well_id = well_id  # The well identifier (e.g., 'A1', 'B1', etc.)

    def bottom(self, offset):
        # Returns the bottom location for the well at a given offset
        return f"Well: {self.well_id} Offset:{offset}"
    def __getitem__(self):
        # This allows indexing with well identifiers like 'A1', 'B1', etc.
        return self.well_id # Return the Well object for the specified well_id

class Plate:
    def __init__(self, plate_type, wells=None):
        self.plate_type = plate_type  # Plate type (96-well, 384-well, etc.)
        
        # Create wells based on plate type
        self.wells = wells or self.create_wells(plate_type)

    def create_wells(self, plate_type):
        # Create well labels for the plate (96-well or 384-well)
        if plate_type == '96-well':
            return {f'{chr(65 + row)}{col + 1}': Well(f'{chr(65 + row)}{col + 1}')
                    for row in range(8) for col in range(12)}  # A1 to H12
        elif plate_type == '384-well':
            return {f'{chr(65 + row)}{col + 1}': Well(f'{chr(65 + row)}{col + 1}')
                    for row in range(16) for col in range(24)}  # A1 to P24
        else:
            raise ValueError("Unsupported plate type. Use '96-well' or '384-well'.")

    def __getitem__(self, well_id):
        # This allows indexing with well identifiers like 'A1', 'B1', etc.
        return self.wells[well_id]  # Return the Well object for the specified well_id
    
frac_plates  = {
    '1': Plate('96-well'),
} 
pool_plates = {
    '1': Plate('384-well'),
}
tips = {
    '1': Plate('96-well'),
} 
buffer = {
    '1': Plate('96-well'),
} 
s300 = S300()

In [5]:
{f'{chr(65 + row)}{col + 1}': Well(f'{chr(65 + row)}{col + 1}')
                    for row in range(8) for col in range(12)}

{'A1': <__main__.Well at 0x7f45d0072c70>,
 'A2': <__main__.Well at 0x7f45d0072820>,
 'A3': <__main__.Well at 0x7f45d0072850>,
 'A4': <__main__.Well at 0x7f45d0072ac0>,
 'A5': <__main__.Well at 0x7f45d0072d30>,
 'A6': <__main__.Well at 0x7f45d0072d60>,
 'A7': <__main__.Well at 0x7f45d00728b0>,
 'A8': <__main__.Well at 0x7f45d0072ca0>,
 'A9': <__main__.Well at 0x7f45c07f1070>,
 'A10': <__main__.Well at 0x7f45c07f1040>,
 'A11': <__main__.Well at 0x7f45c07f1250>,
 'A12': <__main__.Well at 0x7f45c07f1190>,
 'B1': <__main__.Well at 0x7f45d0072df0>,
 'B2': <__main__.Well at 0x7f45c07f1160>,
 'B3': <__main__.Well at 0x7f45d21022b0>,
 'B4': <__main__.Well at 0x7f45d2102610>,
 'B5': <__main__.Well at 0x7f45d2102250>,
 'B6': <__main__.Well at 0x7f45d2102b50>,
 'B7': <__main__.Well at 0x7f45d2102cd0>,
 'B8': <__main__.Well at 0x7f45d21020a0>,
 'B9': <__main__.Well at 0x7f45d2102100>,
 'B10': <__main__.Well at 0x7f45c07d8f10>,
 'B11': <__main__.Well at 0x7f45c07d8f70>,
 'B12': <__main__.Well at 0x7

In [6]:
test = Well('A1')
test.well_id

'A1'

In [7]:
#from opentrons import protocol_api
import numpy as np

#Script originally written by Basile Wicky <bwicky@uw.edu>
#Maintained by Stacey Gerben <srgerb@uw.edu> 
#Let me know if there are bugs

metadata = {
    'protocolName': '### PROTOCOL_NAME ###',
    'author': '### USER_NAME ###',
    'apiLevel':'2.5',
    'description': 'Pool SEC fractions and normalize concentrations (optional)',
}

#======================
# TRANSFER DEFINITIONS
#======================

instrument = 'cwby' # {akta or hplc or cwby} instrument used for SEC -- determines the plate-type of the fractions. Use cwby if normalizing pre-pooled fractions.
destination_plate = '384'
normalize = True # whether or not to normalize concentrations of pooled fractions.
mixing = True # set to False to overwrite the automatic assignemnt of this variable.
debug = False # use for simulation purposes only.
dilutions = '4'
dilution_step = '5'
volume_step = '37.5'

"""
Format of transfer definitions (list of lists):
[
[['frac_1', 'frac_2', ...], [frac_1_vol, frac_2_vol, ...], 'destination_plate', 'destination_well', buffer_volume],
]

Format of frac_x strings:
'akta':'1.A.1'
'hplc':'1-P1-A1'
'cwby':'1_A1'
"""

transfers = np.array([
[ ['1_A1',],[187.5],1,'A1',0],
[ ['1_A2',],[187.5],1,'C1',0],
[ ['1_A3',],[187.5],1,'E1',0],
[ ['1_A4',],[187.5],1,'G1',0],
[ ['1_A5',],[187.5],1,'I1',0],
[ ['1_A6',],[187.5],1,'K1',0],
[ ['1_A7',],[187.5],1,'M1',0],
[ ['1_A8',],[187.5],1,'O1',0],
[ ['1_A9',],[93.75],1,'B1',93.75],
[ ['1_A10',],[93.75],1,'D1',93.75],
[ ['1_A11',],[93.75],1,'F1',93.75],
[ ['1_A12',],[93.75],1,'H1',93.75],
[ ['1_B1',],[93.75],1,'J1',93.75],
[ ['1_B2',],[93.75],1,'L1',93.75],
[ ['1_B3',],[93.75],1,'N1',93.75],
[ ['1_B4',],[93.75],1,'P1',93.75],
[ ['1_B5',],[46.875],1,'A5',140.625],
[ ['1_B6',],[46.875],1,'C5',140.625],
[ ['1_B7',],[46.875],1,'E5',140.625],
[ ['1_B8',],[46.875],1,'G5',140.625],
[ ['1_B9',],[46.875],1,'I5',140.625],
[ ['1_B10',],[46.875],1,'K5',140.625],
[ ['1_B11',],[46.875],1,'M5',140.625],
[ ['1_B12',],[46.875],1,'O5',140.625],
[ ['1_C1',],[23.4375],1,'B5',164.0625],
[ ['1_C2',],[23.4375],1,'D5',164.0625],
[ ['1_C3',],[23.4375],1,'F5',164.0625],
[ ['1_C4',],[23.4375],1,'H5',164.0625],
[ ['1_C5',],[23.4375],1,'J5',164.0625],
[ ['1_C6',],[23.4375],1,'L5',164.0625],
[ ['1_C7',],[23.4375],1,'N5',164.0625],
[ ['1_C8',],[23.4375],1,'P5',164.0625],
[ ['1_C9',],[11.71875],1,'A9',175.78125],
[ ['1_C10',],[11.71875],1,'C9',175.78125],
[ ['1_C11',],[11.71875],1,'E9',175.78125],
[ ['1_C12',],[11.71875],1,'G9',175.78125],
], dtype=object)


In [19]:
sets = int(np.ceil(len(transfers)/8))
sets

5

In [22]:
for i in range(sets):
    for j in range(int(dilutions)):
        print (f'row: {i} dilute:{j}')

row: 0 dilute:0
row: 0 dilute:1
row: 0 dilute:2
row: 0 dilute:3
row: 1 dilute:0
row: 1 dilute:1
row: 1 dilute:2
row: 1 dilute:3
row: 2 dilute:0
row: 2 dilute:1
row: 2 dilute:2
row: 2 dilute:3
row: 3 dilute:0
row: 3 dilute:1
row: 3 dilute:2
row: 3 dilute:3
row: 4 dilute:0
row: 4 dilute:1
row: 4 dilute:2
row: 4 dilute:3


In [8]:
w96 = [r + str(c) for c in range(1,13) for r in 'ABCDEFGH']
print (w96)

['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10', 'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11', 'A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12']


In [9]:
# Global defintions.
z_off = 0.5 # mm -- tip offset from bottom of well.
z_in = 3 # mm -- tip insertion below the liquid surface.
w384 = [r + str(c) for r in 'ABCDEFGHIJKLMNOP' for c in range(1,25)]
w96 = [r + str(c) for r in 'ABCDEFGH' for c in range(1,13)]
if int(destination_plate) == 96:
    plate_format = w96
elif int(destination_plate) == 384:
    plate_format = w384



# # Labware definitions.
# s300 = protocol.load_instrument('p300_single_gen2', 'left') ################################### uncomment!!!!
# s300.flow_rate.aspirate = 300 # uL/s
# s300.flow_rate.dispense = 300 # uL/s
# s300.flow_rate.blow_out = 300 # uL/s

if debug: # use debug plate definitions for simulation purposes only.
    plate_types = {
        'akta':'usascientific_96_wellplate_2.4ml_deep',
        'hplc':'corning_384_wellplate_112ul_flat',
        'cwby':'corning_96_wellplate_360ul_flat',
    }  

else:
    plate_types = {
        'akta':'abgene_96_wellplate_2200ul',
        'hplc':'greiner_384_wellplate_250ul',
        #'cwby':'nunc_96_wellplate_450ul',
        'cwby':'abgene_96_wellplate_2200ul',
    } 
pool_ptype = plate_types['hplc'] if destination_plate == '384' else plate_types['cwby']

# Determine the number of destination plates.
des_plates = np.unique(transfers[:, 2])

# Determine the maximum volume of any well in the pooled plate
if normalize:
    max_pool_vol = np.max([np.sum(t[1]) + t[4] for t in transfers])

else: 
    max_pool_vol = np.max([np.sum(t[1]) for t in transfers])

if max_pool_vol > 200:
    protocol.pause('CAUTION!!! Some pooled wells will overflow while mixing!!!')

# Determine the number of source plates.
if instrument == 'akta':
    src_plates = list(sorted(np.unique([x.split('.')[0] for x in np.hstack(transfers[:, 0])])))

elif instrument == 'hplc':
    src_plates = list(sorted(np.unique(['-'.join(x.split('-')[:2]) for x in np.hstack(transfers[:, 0])])))

elif instrument == 'cwby':
    src_plates = list(sorted(np.unique([x.split('_')[0] for x in np.hstack(transfers[:, 0])])))

else:
    print('Instrument not recognized.')

In [53]:
# # Assign plate-type and plate locations to deck.
# plate2deck = {0:10, 1:11, 2:7, 3:8, 4:4, 5:5}
# frac_ptype = plate_types[instrument]
# frac_plates = {}
# tips = {}
# for i, p_name in enumerate(src_plates):  
#     frac_plates[p_name] = protocol.load_labware(frac_ptype, plate2deck[i], label=f'Fractions {p_name}\n{frac_ptype}')
#     tips[p_name] = protocol.load_labware('opentrons_96_tiprack_300ul', 9-(i*3), label=f'Tips {p_name}')

# pool_plates = {}
# for i, p_name in enumerate(des_plates):
#     pool_plates[p_name] = protocol.load_labware(pool_ptype, i+1, label=f'Destination {p_name}\n{pool_ptype}')

In [54]:
def V2H(vol, z_insert, z_offset, plate):
        '''
        For 96-deepwell plate (AB0932); 39 x 8.4 x 8.4 mm
        For 384-deepwell plate (Greiner 781270); 19.3 x 3.8 x 3.8 mm
        '''
        h_factor = {
                'usascientific_96_wellplate_2.4ml_deep' :0.0177273,
                'corning_384_wellplate_112ul_flat'      :0.0804167,
                'corning_96_wellplate_360ul_flat'       :0.0,
                'abgene_96_wellplate_2200ul'            :0.0177273,
                'greiner_384_wellplate_250ul'           :0.0804167,
                'nunc_96_wellplate_450ul'               :0.0,

        }

        h = h_factor[plate] * vol # height of the liquid.
        h_corr = h - z_insert # height corrected for tip insertion.
        h_actual = h_corr if h_corr > z_offset else z_offset # ensure that min height is z_offset.
        return h_actual

In [55]:
# Add buffer if normalizing concentrations.
if normalize:
    #buffer = protocol.load_labware('agilent_1_reservoir_290ml', 3, label='Buffer') #uncomment later
    s300.pick_up_tip(tips[src_plates[0]]['A1'])
    s300.aspirate(300, buffer['1']['A1'].bottom(4))

    for t in transfers:
        #puts buffer in main well as well as all dilution wells after that
        _, _, des_p, des_w, buffer_vol = t
        well = plate_format.index(des_w)
        for i in range(int(dilutions)):
            print(f'{plate_format[well]}: {buffer_vol}uL')

            # if buffer_vol != 0:
            #     if buffer_vol <= s300.current_volume:
            #         s300.dispense(buffer_vol, pool_plates[str(des_p)][des_w].bottom(V2H(buffer_vol, z_in, z_off, pool_ptype)))
            #     else:
            #         if buffer_vol <= 300: 
            #             s300.aspirate(300-s300.current_volume, buffer['1']['A1'].bottom(4))
            #             s300.dispense(buffer_vol, pool_plates[str(des_p)][des_w].bottom(V2H(buffer_vol, z_in, z_off, pool_ptype)))
            well += 1
            des_w = plate_format[well]
            buffer_vol = float(volume_step) * (float(dilution_step) - 1)

    s300.blow_out(buffer['A1'])
    s300.return_tip()

Picking up tip at <__main__.Well object at 0x7fc87ae8ccd0>
Aspirating 300uL from Well: A1 Offset:4
A1: 0uL
A2: 150.0uL
A3: 150.0uL
A4: 150.0uL
C1: 0uL
C2: 150.0uL
C3: 150.0uL
C4: 150.0uL
E1: 0uL
E2: 150.0uL
E3: 150.0uL
E4: 150.0uL
G1: 0uL
G2: 150.0uL
G3: 150.0uL
G4: 150.0uL
I1: 0uL
I2: 150.0uL
I3: 150.0uL
I4: 150.0uL
K1: 0uL
K2: 150.0uL
K3: 150.0uL
K4: 150.0uL
M1: 0uL
M2: 150.0uL
M3: 150.0uL
M4: 150.0uL
N1: 0uL
N2: 150.0uL
N3: 150.0uL
N4: 150.0uL
O1: 93.75uL
O2: 150.0uL
O3: 150.0uL
O4: 150.0uL
B1: 93.75uL
B2: 150.0uL
B3: 150.0uL
B4: 150.0uL
D1: 93.75uL
D2: 150.0uL
D3: 150.0uL
D4: 150.0uL
F1: 93.75uL
F2: 150.0uL
F3: 150.0uL
F4: 150.0uL
H1: 93.75uL
H2: 150.0uL
H3: 150.0uL
H4: 150.0uL
J1: 93.75uL
J2: 150.0uL
J3: 150.0uL
J4: 150.0uL
L1: 93.75uL
L2: 150.0uL
L3: 150.0uL
L4: 150.0uL
N1: 93.75uL
N2: 150.0uL
N3: 150.0uL
N4: 150.0uL
P5: 140.625uL
P6: 150.0uL
P7: 150.0uL
P8: 150.0uL
A5: 140.625uL
A6: 150.0uL
A7: 150.0uL
A8: 150.0uL
C5: 140.625uL
C6: 150.0uL
C7: 150.0uL
C8: 150.0uL
E5: 140.625uL
E

KeyError: 'A1'